In [1]:
import pandas as pd
import numpy as np
from os.path import join as PJOIN
import os

In [2]:
PREFIX = "comments"
PROJECT_NAME = "GoogleTest"
SUFFIX = "all_marked"
#FILE_NAME = PREFIX + "_" + PROJECT_NAME + "_" + SUFFIX + ".xlsx"
#annotation sheet file path
FILE_PATH = "DATA/ANNOTATED/annotation_gtest.csv"
#FILE_PATH = PJOIN("DATA","ANNOTATED",FILE_NAME)
OUTPUT_FILE_PATH = PJOIN("DATA","GENERATED",os.path.basename(FILE_PATH))
MAP = {'U':'U', 'PU':'P', 'NU':'N'}
THRESHOLD = 10
ANNOTATION_CLASS_START = 16
ANNOTATION_CLASS_NUM = 30

In [3]:
rules = []

rules.append((lambda c:(c[9] or c[11]) and not (c[1] or c[2] or c[4] or c[5]),'U',1))
rules.append((lambda c:(c[9] or c[11]) and (c[4] or c[5]),'PU',2))
rules.append((lambda c:(c[9] or c[11]) and (c[1] or c[2]), 'NU',3))
rules.append((lambda c:c[10] and not (c[1] or c[2] or c[4] or c[5]) and c[15],'U',4))
rules.append((lambda c:c[10] and not (c[1] or c[2] or c[4] or c[5]) and c[14],'PU',5))
rules.append((lambda c:c[8] and not (c[7] or c[10] or c[11]) and not (c[1] or c[2] or c[4] or c[5]) and c[15],'PU',6))
rules.append((lambda c:c[8] and not (c[7] or c[10] or c[11]) and not (c[1] or c[2] or c[4] or c[5]) and c[14],'NU',7))
rules.append((lambda c:c[10] and (c[1] or c[2]),'NU',8))
rules.append((lambda c:c[10] and  (c[4] or c[5]) and c[15],'PU',9))
rules.append((lambda c:(c[10] or c[8]) and (c[4] or c[5]) and c[14],'NU',10))
rules.append((lambda c:c[8] and  (c[1] or c[2] or c[4] or c[5]),'NU',11))
rules.append((lambda c:c[6] and c[7],'NU',12))
rules.append((lambda c:c[10] and c[9] and  (c[4] or c[5]) and c[14],'PU',13))
rules.append((lambda c:c[12],'NU',14))
rules.append((lambda c:c[1] or c[2],'NU',15))

rules.append((lambda c:c[16] or c[17],'PU',16))
rules.append((lambda c:c[18] or c[19] or c[21],'PU',17))
rules.append((lambda c:c[20],'U',18))
rules.append((lambda c:c[22],'U',19))
rules.append((lambda c:c[23] or c[24],'PU',20))
rules.append((lambda c:c[25] or c[29] or c[26],'U',21))
rules.append((lambda c:c[27] or c[28],'PU',22))
rules.append((lambda c:c[30],'NU',23))
# rules.append((lambda c:))

# Rules

In [4]:
# c is a vector of size 31 [comment text, C1, C2, ......., C30, C31]
CTS0CT = 0
def get_label_all(c): #FOR ANALYSIS ONLY
    labels = []
    for rule in rules:
        if rule[0](c):
            labels.append([rule[1],rule[2]])
    return labels

# Information content - any Useful -> useful
#                     - only NU - Later PU -> or NU -> NU
#                               - Later U -> U
# Information content - atlet 1 PU - Later atleat 1 PU -> PU
#                                  - Later atleast 1 U -> U

def get_matching_rules(c):
    res = []
    for i in range(23):
        if rules[i][0](c):
            res.append(rules[i][2])
    res = [str(x) for x in res]
    return res
    
def get_label(c):
    global CTS0CT
    cts = {'U':0,'PU':0,'NU':0}
    for i in range(15):
        if rules[i][0](c):
            cts[rules[i][1]] += 1
    
    cts_late = {'U':0,'PU':0,'NU':0}
    for i in range(15, len(rules)):
        if rules[i][0](c):
            cts_late[rules[i][1]] += 1
    
#     if cts['U'] == 0 and cts['PU'] == 0 and cts['NU'] == 0:
#         CTS0CT += 1
#         print(cts_late)
    if cts['U'] > 0:
        return 'U'
    if cts['PU'] > 0 or cts['NU'] == 0:
        if cts_late['U'] > 0: 
            return 'U'
        
        return 'PU'
    
    if cts_late['U'] > 0 or cts_late['PU'] >0:
        return 'PU'
    return 'NU'
    
    
            
    if cts['PU'] > 0:
        if cts_late['U'] > 0:
            return 'U'
        if cts_late['PU'] > 0 :
            return 'PU'
        return 'NU'
    else:
        if cts_late['U'] > 0:
            return 'PU'
        if cts_late['NU'] > 0:
            return 'NU'
        return 'NU'
        
        

def get_label_old(c):
    #IF C18 OR C19 OR C20  OR  C21  OR C22 OR C28 OR C29 THEN  U
    if c[18] or c[19] or c[20] or c[21] or c[22] or c[28] or c[29]:
        return 'U'
    #IF C9 AND C3 THEN U
    if c[9] and c[3]:
        return 'U'
    #IF C11 AND C3 THEN U
    if c[11] and c[3]:
        return 'U'
    #IF (C25 OR C23 OR C26 OR C27 ) AND C3 THEN U
    if (c[25] or c[23] or c[26] or c[27]) and c[3]:
        return 'U'
    #IF (C25 OR C23 OR C26 OR C27) AND C4 AND (C9  OR C11) THEN U
    if (c[25] or c[23] or c[26] or c[27]) and c[4] and (c[9] or c[11]):
        return 'U'
    #IF C10 AND C15  AND C3 THEN U
    if c[10] and c[15] and c[3]:
        return 'U'
    #IF C8 AND C15  AND C3 THEN U
    if c[8] and c[15] and c[3]:
        return 'U'
    #IF (C18 OR C19 OR C20 OR C21) AND C17 THEN U
    if (c[18] or c[19] or c[20] or c[21]) and c[17]:
        return 'U'
    #Low Problem Domain AND Low Scope AND Concepts don't match Symbols = U
    if c[10] and c[15] and c[3]:
        return 'U'
    #Low Problem Domain AND Low Scope AND Concepts Partially Match = U
    if c[10] and c[15] and c[4]:
        return 'U'
    
    #IF C9 AND (C4 OR C5) THEN PU
    if c[9] and (c[4] or c[5]):
        return 'PU'
    #IF C11 AND (C4  OR C5) THEN PU
    if c[11] and (c[4] or c[5]):
        return 'PU'
    #IF C10 AND C14  AND C3 THEN PU
    if c[1] and c[14] and c[3]:
        return 'PU'
    #IF C8 AND C14  AND C3 THEN PU
    if c[8] and c[14] and c[3]:
        return 'PU'
    #IF C17 AND NOT (C18 OR C19 OR C20 OR C21)  THEN PU
    if c[17] and not((c[18] or c[19] or c[20] or c[21])):
        return 'PU'
    #Low  Problem Domain AND Concepts Partially Match = PU
    if c[10] and c[4]:
        return 'PU'
    #Low Program Domain AND Concepts Partially Match = PU
    if c[8] and c[4]:
        return 'PU'
    
    #IF C12 OR C16 THEN  NU
    if c[12] or c[16]:
        return 'NU'
    #IF (C8 OR C9)  AND C1 THEN NU
    if (c[8] or c[9]) and c[1]:
        return 'NU'
    #IF (C10 OR C11) AND C1 THEN NU
    if (c[10] or c[11]) and c[1]:
        return 'NU'
    #IF C10 AND C15  AND C1 THEN U
    if c[10] and c[15] and c[1]:
        return 'NU'
    #IF C8 AND C15  AND C1 THEN U
    if c[8] and c[15] and c[1]:
        return 'NU'
    
    #print("###NOMATCH")
    return 'NU'

#return multilabel classification labels 
#[Program Domain(non redundant), Program Domain (redundant), Problem Domain (NR), Problem Domain (R), 
#Project Management]
def get_label_comment_classification(c):
    res = [0,0,0]
    if c[8] or c[9]:
        res[0] = 1
    if c[10] or c[11]:
        res[1] = 1
    if np.any(c[17:30]):
        res[2] = 1
    
    return res
    
        


In [5]:
if FILE_PATH[-3:] == 'csv':
    exl_file = pd.read_csv(FILE_PATH,delimiter='$')
else:
    exl_file = pd.read_excel(FILE_PATH)
exl_file.head()

,Filename,Comment text,Start line,End line,No. of words,Program Domain Concepts,Problem Domain Concepts,Copyright/License,Bug/Fix/Patch/Version,Build,...,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30
0,googletest/googletest/src/gtest-all.cc/gtest-a...,"Copyright 2008, Google Inc.\nAll rights reserv...",1,28,222,"{'rights': ['right', 'Data-Structure and its C...","['event', 'google']",True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,googletest/googletest/src/gtest-all.cc/gtest-a...,\nGoogle C++ Testing and Mocking Framework (Go...,30,34,25,"{'testing': ['reseting', 'Operations as part o...","['file', 'mock', 'google', 'test']",False,False,True,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,googletest/googletest/src/gtest-all.cc/gtest-a...,This line ensures that gtest.h can be compiled...,36,37,15,"{'ensures': ['enquires', 'Operations as part o...","['gtest', 'test']",False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,googletest/googletest/src/gtest-all.cc/gtest-a...,The following lines pull in the real gtest *.c...,40,40,10,"{'real': ['real', 'Data-Structure and its Comp...","['files', 'file', 'gtest', 'test']",False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,googletest/googlemock/src/gmock-all.cc/gmock-a...,"Copyright 2008, Google Inc.\nAll rights reserv...",1,28,222,"{'rights': ['right', 'Data-Structure and its C...","['event', 'google']",True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
exl_np = np.array(exl_file)
for i,el in enumerate(exl_np[0]):
    print(i," : ",el)

0  :  googletest/googletest/src/gtest-all.cc/gtest-all.cc
1  :  Copyright 2008, Google Inc.
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are
met:

* Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.
* Redistributions in binary form must reproduce the above
copyright notice, this list of conditions and the following disclaimer
in the documentation and/or other materials provided with the
distribution.
* Neither the name of Google Inc. nor the names of its
contributors may be used to endorse or promote products derived from
this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE

In [7]:
classes = exl_np[:,[1]+list(range(ANNOTATION_CLASS_START,ANNOTATION_CLASS_START + ANNOTATION_CLASS_NUM))]
classes[0]

array(['Copyright 2008, Google Inc.\nAll rights reserved.\n\nRedistribution and use in source and binary forms, with or without\nmodification, are permitted provided that the following conditions are\nmet:\n\n* Redistributions of source code must retain the above copyright\nnotice, this list of conditions and the following disclaimer.\n* Redistributions in binary form must reproduce the above\ncopyright notice, this list of conditions and the following disclaimer\nin the documentation and/or other materials provided with the\ndistribution.\n* Neither the name of Google Inc. nor the names of its\ncontributors may be used to endorse or promote products derived from\nthis software without specific prior written permission.\n\nTHIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS\n"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT\nLIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR\nA PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE C

In [8]:
for j, c in enumerate(classes):
    for i in range(1,31):
        try:
            if c[i] != c[i]:
                c[i] = False
#             elif c[i] == 'False':
#                 c[i] = False
#             elif c[i] == 'True':
#                 c[i] = True
            elif c[i] == ' ' or c[i] == '\n' or c[i] == 'False':
                c[i] = False
            elif int(float(c[i])) == 1:
                c[i] = True
            else:
                c[i] = False
        except Exception as e:
            print(j, i, e)

In [11]:
classes[6]

array(["This line ensures that gmock.h can be compiled on its own, even\nwhen it's fused.",
       False, False, False, False, False, False, False, True, False, True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False], dtype=object)

In [12]:
labels = []
labels_comment_classification = []
matching_rules = []
for c in classes:
    labels.append(MAP[get_label(c)])
    matching_rules.append(','.join(get_matching_rules(c)))
    labels_comment_classification.append(get_label_comment_classification(c))

In [13]:
exl_file['Calculated Score New'] = labels
exl_file['Matching Rules'] = matching_rules
exl_file['Comment Type'] = labels_comment_classification

In [14]:
if FILE_PATH[-3:] == 'csv':
    exl_file.to_csv(OUTPUT_FILE_PATH,index=False,sep='$')
else:
    exl_file.to_excel(OUTPUT_FILE_PATH, index=False)

# Analysis

In [15]:
intuitive_labels = exl_np[:,-1]

In [16]:
lables_np = np.array(labels)

In [172]:
num_nan = np.sum(np.array([x !=x for x in intuitive_labels]))

In [173]:
sum(lables_np == intuitive_labels)/(len(lables_np) - num_nan), (len(lables_np) - num_nan)

(0.08846584546472565, 893)

In [174]:
losses = {}

In [175]:
for i in range(len(intuitive_labels)):
    if intuitive_labels[i]!=intuitive_labels[i]:
        continue
    li = intuitive_labels[i]
    lc = lables_np[i]
    if li not in losses:
        losses[li] = {}
    if lc not in losses[li]:
        losses[li][lc] = 0
    losses[li][lc] += 1

In [176]:
losses

{'Pu': {'N': 125, 'P': 6, 'U': 21},
 'U': {'N': 112, 'P': 41, 'U': 79},
 'Us': {'N': 507, 'P': 2}}

In [177]:
sm = 0
for i in range(len(intuitive_labels)):
    if intuitive_labels[i] != intuitive_labels[i]:
        continue
    if intuitive_labels[i][0] == lables_np[i][0]:
        sm+=1

In [178]:
sm/(len(lables_np) - num_nan)

0.09518477043673013

In [17]:
losses = {}
def get_hash(classes):
    hsh = []
    for i, c in enumerate(classes):
        if c == True:
            hsh.append(i+1)
    return str(hsh)

for i in range(len(intuitive_labels)):
    if intuitive_labels[i]!=intuitive_labels[i]:
        continue
    li = intuitive_labels[i][0]
    lc = lables_np[i]
    if li not in losses:
        losses[li] = {}
    if lc not in losses[li]:
        losses[li][lc] = {}
    if li==lc:
        continue
    myhash = get_hash(classes[i][1:])
    if myhash not in losses[li][lc]:
        losses[li][lc][myhash] = 0
    losses[li][lc][myhash] += 1

In [18]:
alls = []
for li, v1 in losses.items():
    for lc, v2 in v1.items():
        curr = 0
        tops_curr = None
        for  myhash, ct in v2.items():
            alls.append([li,lc,myhash,ct])
#             if ct > curr:
#                 curr = ct
#                 tops_curr = [li, lc, myhash, ct]
#         if tops_curr is not None:
#             tops.append(tops_curr)

In [19]:
print("Inutinitive Label | Calculated Label | Classes which have 1 | Number of such mismatch")
alls = sorted(alls,key=lambda x:(x[3],x[0],x[1]),reverse=True)
alls

Inutinitive Label | Calculated Label | Classes which have 1 | Number of such mismatch


[['U', 'N', '[3, 7, 8]', 114],
 ['U', 'N', '[3, 6, 7, 13]', 66],
 ['U', 'N', '[3, 8, 10]', 61],
 ['P', 'N', '[3, 7, 8]', 59],
 ['N', 'U', '[3, 7, 8, 13, 15]', 57],
 ['P', 'N', '[3, 7, 8, 13]', 51],
 ['P', 'U', '[3, 7, 9]', 40],
 ['P', 'N', '[3, 6, 7, 13]', 40],
 ['N', 'U', '[2, 3, 7, 8, 13, 15]', 39],
 ['U', 'N', '[3, 7, 8, 13]', 33],
 ['P', 'N', '[3, 6, 7, 13, 15]', 25],
 ['P', 'N', '[1, 6, 7, 13, 15]', 25],
 ['P', 'U', '[3, 7, 8, 13, 15]', 19],
 ['U', 'N', '[3, 6, 7, 13, 15]', 17],
 ['U', 'N', '[3, 6, 10]', 17],
 ['U', 'P', '[4, 7, 9]', 16],
 ['U', 'N', '[3, 6, 7]', 16],
 ['U', 'P', '[4, 9, 11]', 13],
 ['U', 'N', '[3, 6, 10, 13]', 13],
 ['P', 'U', '[3, 9, 10]', 12],
 ['U', 'P', '[4, 9, 10]', 11],
 ['U', 'N', '[1, 7, 8, 15]', 10],
 ['P', 'N', '[3, 8, 10]', 10],
 ['U', 'P', '[4, 9, 11, 14]', 9],
 ['P', 'U', '[3, 6, 10, 13, 15]', 9],
 ['P', 'N', '[3, 6, 10, 13]', 8],
 ['U', 'N', '[3, 8, 10, 13]', 7],
 ['P', 'U', '[3, 7, 8, 15]', 7],
 ['P', 'N', '[1, 6, 10, 13, 15]', 7],
 ['N', 'U', '[3,

In [20]:
import csv
with open('DATA/ANNOTATED/rule_diffs/'+PROJECT_NAME+".csv",'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Intuitive Label","Calculated Label", "Classes which have 1 marked","Number of Mismatches", "Name"])
    for i, row in enumerate(alls):
        if row[3] < THRESHOLD:
            break
        writer.writerow(row + ["Case "+str(i+1)])

In [21]:
alls

[['U', 'N', '[3, 7, 8]', 114],
 ['U', 'N', '[3, 6, 7, 13]', 66],
 ['U', 'N', '[3, 8, 10]', 61],
 ['P', 'N', '[3, 7, 8]', 59],
 ['N', 'U', '[3, 7, 8, 13, 15]', 57],
 ['P', 'N', '[3, 7, 8, 13]', 51],
 ['P', 'U', '[3, 7, 9]', 40],
 ['P', 'N', '[3, 6, 7, 13]', 40],
 ['N', 'U', '[2, 3, 7, 8, 13, 15]', 39],
 ['U', 'N', '[3, 7, 8, 13]', 33],
 ['P', 'N', '[3, 6, 7, 13, 15]', 25],
 ['P', 'N', '[1, 6, 7, 13, 15]', 25],
 ['P', 'U', '[3, 7, 8, 13, 15]', 19],
 ['U', 'N', '[3, 6, 7, 13, 15]', 17],
 ['U', 'N', '[3, 6, 10]', 17],
 ['U', 'P', '[4, 7, 9]', 16],
 ['U', 'N', '[3, 6, 7]', 16],
 ['U', 'P', '[4, 9, 11]', 13],
 ['U', 'N', '[3, 6, 10, 13]', 13],
 ['P', 'U', '[3, 9, 10]', 12],
 ['U', 'P', '[4, 9, 10]', 11],
 ['U', 'N', '[1, 7, 8, 15]', 10],
 ['P', 'N', '[3, 8, 10]', 10],
 ['U', 'P', '[4, 9, 11, 14]', 9],
 ['P', 'U', '[3, 6, 10, 13, 15]', 9],
 ['P', 'N', '[3, 6, 10, 13]', 8],
 ['U', 'N', '[3, 8, 10, 13]', 7],
 ['P', 'U', '[3, 7, 8, 15]', 7],
 ['P', 'N', '[1, 6, 10, 13, 15]', 7],
 ['N', 'U', '[3,

# Extend Annotations with Diff

In [22]:
alls_map = {}
for i, row in enumerate(alls):
    if row[3] < THRESHOLD:
        break
    alls_map[(row[0],row[1],row[2])] = i+1
alls_map

{('N', 'U', '[2, 3, 7, 8, 13, 15]'): 9,
 ('N', 'U', '[3, 7, 8, 13, 15]'): 5,
 ('P', 'N', '[1, 6, 7, 13, 15]'): 12,
 ('P', 'N', '[3, 6, 7, 13, 15]'): 11,
 ('P', 'N', '[3, 6, 7, 13]'): 8,
 ('P', 'N', '[3, 7, 8, 13]'): 6,
 ('P', 'N', '[3, 7, 8]'): 4,
 ('P', 'N', '[3, 8, 10]'): 23,
 ('P', 'U', '[3, 7, 8, 13, 15]'): 13,
 ('P', 'U', '[3, 7, 9]'): 7,
 ('P', 'U', '[3, 9, 10]'): 20,
 ('U', 'N', '[1, 7, 8, 15]'): 22,
 ('U', 'N', '[3, 6, 10, 13]'): 19,
 ('U', 'N', '[3, 6, 10]'): 15,
 ('U', 'N', '[3, 6, 7, 13, 15]'): 14,
 ('U', 'N', '[3, 6, 7, 13]'): 2,
 ('U', 'N', '[3, 6, 7]'): 17,
 ('U', 'N', '[3, 7, 8, 13]'): 10,
 ('U', 'N', '[3, 7, 8]'): 1,
 ('U', 'N', '[3, 8, 10]'): 3,
 ('U', 'P', '[4, 7, 9]'): 16,
 ('U', 'P', '[4, 9, 10]'): 21,
 ('U', 'P', '[4, 9, 11]'): 18}

In [23]:
diff_names = []
diffs_map = {}
for i in range(len(intuitive_labels)):
    if intuitive_labels[i]!=intuitive_labels[i]:
        diff_names.append(None)
        continue
    li = intuitive_labels[i][0]
    lc = lables_np[i]
    myhash = get_hash(classes[i][1:])
    if (li, lc, myhash) not in alls_map:
        diff_names.append(None)
        continue
    diff_names.append("Case " + str(alls_map[(li, lc, myhash)]))
    if alls_map[(li, lc, myhash)] not in diffs_map:
        diffs_map[alls_map[(li, lc, myhash)]] = []
    diffs_map[alls_map[(li, lc, myhash)]].append(i)

In [24]:
exl_file['Mismatch Case'] = diff_names

In [25]:
DIFFS_FILE = FILE_NAME
if DIFFS_FILE[-3:] == 'csv':
    DIFFS_FILE = DIFFS_FILE[:-3] + 'xlsx'

In [26]:
writer = pd.ExcelWriter("DATA/ANNOTATED/rule_diffs/"+DIFFS_FILE, engine = 'xlsxwriter')

In [27]:
exl_file.to_excel(writer,index=False)

In [28]:
for k,v in diffs_map.items():
    df = pd.DataFrame(columns=exl_file.columns)
    for i, j in enumerate(v):
        df.loc[i] = exl_file.loc[j]
    df.to_excel(writer,index=False,sheet_name='Case '+str(k))

In [29]:
writer.save()
writer.close()

# Counts of CLasses

In [21]:
TOTAL = len(classes)
print("TOATAL - ",TOTAL)

TOATAL -  6227


In [14]:
SUMS = np.sum(classes==True,axis=0)
for i in range(17,30):
    print("Class ",i, " -> ",SUMS[i],"\t\t",'{0:.2f}%'.format(float(SUMS[i]*100./TOTAL)))

Class  17  ->  37 		 0.59%
Class  18  ->  4 		 0.06%
Class  19  ->  0 		 0.00%
Class  20  ->  120 		 1.93%
Class  21  ->  23 		 0.37%
Class  22  ->  110 		 1.77%
Class  23  ->  25 		 0.40%
Class  24  ->  3 		 0.05%
Class  25  ->  19 		 0.31%
Class  26  ->  46 		 0.74%
Class  27  ->  81 		 1.30%
Class  28  ->  34 		 0.55%
Class  29  ->  67 		 1.08%


In [25]:
c8_11_prev_0 = 0
c8_9_prev_0 = 0
c10_11_prev_0 = 0

def makeKey(vals):
    res = []
    for val in vals:
        if val:
            res.append(1)
        else:
            res.append(0)
    return str(res)

MMP = {}
for c in classes:
    if np.any(c[8:12]) and not(np.any(c[[1,2,4,5]])):
        c8_11_prev_0 += 1
        k = makeKey(c[8:12])
        if k not in MMP:
            MMP[k] = 0
        MMP[k] += 1
    if np.any(c[8:10]) and not(np.any(c[[1,2,4,5]])):
        c8_9_prev_0 += 1
    if np.any(c[10:12]) and not(np.any(c[[1,2,4,5]])):
        c10_11_prev_0 += 1

In [26]:
print(c8_11_prev_0,"\t\t",'{0:.2f}%'.format(float(c8_11_prev_0*100./TOTAL)))
print(c8_9_prev_0,"\t\t",'{0:.2f}%'.format(float(c8_9_prev_0*100./TOTAL)))
print(c10_11_prev_0,"\t\t",'{0:.2f}%'.format(float(c10_11_prev_0*100./TOTAL)))

4923 		 79.06%
4585 		 73.63%
2065 		 33.16%


In [37]:
MMP

{'[0, 1, 0, 1]': 411,
 '[0, 1, 1, 0]': 469,
 '[1, 0, 0, 1]': 101,
 '[1, 0, 1, 0]': 303}

In [28]:
np.sum(np.any(labels_comment_classification,axis=1))/TOTAL

0.8389272522884214

In [54]:
labels = []
for c in classes:
    labels.append(get_label(c))

In [55]:
labels

[[['NU', 12], ['PU', 15], ['PU', 16]],
 [['NU', 11], ['PU', 21]],
 [['NU', 12], ['PU', 15], ['PU', 16]],
 [['PU', 21]],
 [['NU', 11]],
 [['PU', 21]],
 [['NU', 11]],
 [['NU', 11]],
 [['PU', 2], ['U', 17], ['U', 20]],
 [['U', 17]],
 [['NU', 11], ['U', 17]],
 [['PU', 2], ['U', 20]],
 [['PU', 19]],
 [['PU', 19]],
 [['NU', 11], ['U', 17], ['U', 20]],
 [],
 [['NU', 11]],
 [['NU', 11]],
 [['NU', 3], ['PU', 19], ['U', 20]],
 [['NU', 12], ['NU', 22]],
 [['NU', 12], ['NU', 22]],
 [['NU', 11], ['PU', 19], ['U', 20]],
 [['NU', 11], ['U', 20]],
 [['NU', 11], ['U', 20]],
 [['NU', 10], ['NU', 11], ['U', 20]],
 [['NU', 11], ['U', 20]],
 [['PU', 2], ['NU', 10], ['PU', 13], ['U', 20], ['PU', 21]],
 [['PU', 2], ['NU', 10], ['PU', 13], ['U', 20]],
 [['PU', 2], ['PU', 9], ['U', 20]],
 [['PU', 2], ['PU', 9], ['U', 20]],
 [['NU', 11]],
 [['NU', 12], ['PU', 15], ['PU', 16]],
 [['NU', 12], ['PU', 16], ['NU', 22]],
 [['NU', 11], ['PU', 21]],
 [['NU', 11]],
 [['PU', 21]],
 [['NU', 11], ['PU', 21]],
 [['NU', 11]]

In [11]:
np.sum(np.array(labels) == 'N'), np.sum(np.array(labels) == 'U'), np.sum(np.array(labels) == 'P')

(4049, 8250, 6735)

In [234]:
labels = labels[:1500]

In [255]:
int('1.0')

ValueError: invalid literal for int() with base 10: '1.0'

In [321]:
for c in classes:
    if not np.any(c[1:32]):
        print(c)

In [337]:
CTS0CT

4330

In [22]:
ct = {}
for el in labels:
    if el not in ct:
        ct[el] = 0
    ct[el] += 1
ct

{'N': 1188, 'P': 1456, 'U': 2528}